
<br>
Agglomerative clustering with different metrics<br>
===============================================<br>
Demonstrates the effect of different metrics on the hierarchical clustering.<br>
The example is engineered to show the effect of the choice of different<br>
metrics. It is applied to waveforms, which can be seen as<br>
high-dimensional vector. Indeed, the difference between metrics is<br>
usually more pronounced in high dimension (in particular for euclidean<br>
and cityblock).<br>
We generate data from three groups of waveforms. Two of the waveforms<br>
(waveform 1 and waveform 2) are proportional one to the other. The cosine<br>
distance is invariant to a scaling of the data, as a result, it cannot<br>
distinguish these two waveforms. Thus even with no noise, clustering<br>
using this distance will not separate out waveform 1 and 2.<br>
We add observation noise to these waveforms. We generate very sparse<br>
noise: only 6% of the time points contain noise. As a result, the<br>
l1 norm of this noise (ie "cityblock" distance) is much smaller than it's<br>
l2 norm ("euclidean" distance). This can be seen on the inter-class<br>
distance matrices: the values on the diagonal, that characterize the<br>
spread of the class, are much bigger for the Euclidean distance than for<br>
the cityblock distance.<br>
When we apply clustering to the data, we find that the clustering<br>
reflects what was in the distance matrices. Indeed, for the Euclidean<br>
distance, the classes are ill-separated because of the noise, and thus<br>
the clustering does not separate the waveforms. For the cityblock<br>
distance, the separation is good and the waveform classes are recovered.<br>
Finally, the cosine distance does not separate at all waveform 1 and 2,<br>
thus the clustering puts them in the same cluster.<br>
<br>
Author: Gael Varoquaux<br>
License: BSD 3-Clause or CC-0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances

In [ ]:
np.random.seed(0)

Generate waveform data

In [ ]:
n_features = 2000
t = np.pi * np.linspace(0, 1, n_features)

In [ ]:
def sqr(x):
    return np.sign(np.cos(x))

In [ ]:
X = list()
y = list()
for i, (phi, a) in enumerate([(.5, .15), (.5, .6), (.3, .2)]):
    for _ in range(30):
        phase_noise = .01 * np.random.normal()
        amplitude_noise = .04 * np.random.normal()
        additional_noise = 1 - 2 * np.random.rand(n_features)
        # Make the noise sparse
        additional_noise[np.abs(additional_noise) < .997] = 0
        X.append(12 * ((a + amplitude_noise)
                 * (sqr(6 * (t + phi + phase_noise)))
                 + additional_noise))
        y.append(i)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
n_clusters = 3

In [ ]:
labels = ('Waveform 1', 'Waveform 2', 'Waveform 3')

Plot the ground-truth labelling

In [ ]:
plt.figure()
plt.axes([0, 0, 1, 1])
for l, c, n in zip(range(n_clusters), 'rgb',
                   labels):
    lines = plt.plot(X[y == l].T, c=c, alpha=.5)
    lines[0].set_label(n)

In [ ]:
plt.legend(loc='best')

In [ ]:
plt.axis('tight')
plt.axis('off')
plt.suptitle("Ground truth", size=20)

Plot the distances

In [ ]:
for index, metric in enumerate(["cosine", "euclidean", "cityblock"]):
    avg_dist = np.zeros((n_clusters, n_clusters))
    plt.figure(figsize=(5, 4.5))
    for i in range(n_clusters):
        for j in range(n_clusters):
            avg_dist[i, j] = pairwise_distances(X[y == i], X[y == j],
                                                metric=metric).mean()
    avg_dist /= avg_dist.max()
    for i in range(n_clusters):
        for j in range(n_clusters):
            plt.text(i, j, '%5.3f' % avg_dist[i, j],
                     verticalalignment='center',
                     horizontalalignment='center')
    plt.imshow(avg_dist, interpolation='nearest', cmap=plt.cm.gnuplot2,
               vmin=0)
    plt.xticks(range(n_clusters), labels, rotation=45)
    plt.yticks(range(n_clusters), labels)
    plt.colorbar()
    plt.suptitle("Interclass %s distances" % metric, size=18)
    plt.tight_layout()

Plot clustering results

In [ ]:
for index, metric in enumerate(["cosine", "euclidean", "cityblock"]):
    model = AgglomerativeClustering(n_clusters=n_clusters,
                                    linkage="average", affinity=metric)
    model.fit(X)
    plt.figure()
    plt.axes([0, 0, 1, 1])
    for l, c in zip(np.arange(model.n_clusters), 'rgbk'):
        plt.plot(X[model.labels_ == l].T, c=c, alpha=.5)
    plt.axis('tight')
    plt.axis('off')
    plt.suptitle("AgglomerativeClustering(affinity=%s)" % metric, size=20)

In [ ]:
plt.show()